In [1]:
# ============================
# Project 2: Load CSVs into Render PostgreSQL
# ============================

# Step 0️⃣: Imports
import pandas as pd
import psycopg2

# ------------------------
# Step 1️⃣: Connect to Render DB
# ------------------------
conn = psycopg2.connect(
    host="dpg-d4ltqca4d50c73e9pu8g-a.oregon-postgres.render.com",
    database="mini_project_2_ga5e",
    user="mini_project_2_ga5e_user",
    password="n1JnTZoEbDnU982DU5aVpWwQl2EJXlF4",
    port=5432,
    sslmode="require"
)
cursor = conn.cursor()
print("Connected to Render PostgreSQL ✅")

# ------------------------
# Step 2️⃣: Clear existing tables (optional)
# ------------------------
tables = ["OrderDetail", "Product", "ProductCategory", "Customer", "Country", "Region"]
for t in tables:
    cursor.execute(f"TRUNCATE TABLE {t} CASCADE;")
conn.commit()
print("Tables truncated ✅")

# ------------------------
# Step 3️⃣: Generic CSV loader for small tables
# ------------------------
def load_csv_with_id(csv_file, table_name):
    df = pd.read_csv(csv_file)
    rows = list(df.itertuples(index=False, name=None))
    placeholders = ",".join(["%s"] * len(df.columns))
    sql = f"INSERT INTO {table_name} VALUES ({placeholders})"
    cursor.executemany(sql, rows)
    conn.commit()
    print(f"Loaded {len(rows)} rows into {table_name}!")

Connected to Render PostgreSQL ✅
Tables truncated ✅


In [4]:
# ------------------------
# Step 4️⃣: Load small tables
# ------------------------
load_csv_with_id("Region.csv", "Region")
load_csv_with_id("Country.csv", "Country")
load_csv_with_id("Customer.csv", "Customer")

Loaded 9 rows into Region!
Loaded 21 rows into Country!
Loaded 91 rows into Customer!


In [6]:
load_csv_with_id("ProductCategory.csv", "ProductCategory")
load_csv_with_id("Product.csv", "Product")
print("Small tables loaded ✅")


Loaded 8 rows into ProductCategory!
Loaded 77 rows into Product!
Small tables loaded ✅


In [8]:
# ------------------------
# Step 5️⃣: Prepare OrderDetail CSV for fast load
# ------------------------
# Drop OrderID so PostgreSQL can auto-generate it
df_od = pd.read_csv("OrderDetail.csv")
df_od = df_od.drop(columns=["OrderID"])
df_od.to_csv("OrderDetail_noID.csv", index=False)
print("OrderDetail CSV prepared without OrderID ✅")


OrderDetail CSV prepared without OrderID ✅


In [10]:
# ------------------------
# Step 6️⃣: Fast load OrderDetail using COPY
# ------------------------
with open("OrderDetail_noID.csv", "r") as f:
    cursor.copy_expert("""
        COPY OrderDetail(CustomerID, ProductID, OrderDate, QuantityOrdered)
        FROM STDIN WITH CSV HEADER
    """, f)

conn.commit()
print("OrderDetail loaded via COPY ✅")

OrderDetail loaded via COPY ✅


In [19]:
# Quickly check the counts in all tables:

tables = ["Region", "Country", "Customer", "ProductCategory", "Product", "OrderDetail"]

for t in tables:
    cursor = conn.cursor()
    cursor.execute(f"SELECT COUNT(*) FROM {t};")
    count = cursor.fetchone()[0]
    print(f"{t}: {count} rows")
    cursor.close()


Region: 9 rows
Country: 21 rows
Customer: 91 rows
ProductCategory: 8 rows
Product: 77 rows
OrderDetail: 621806 rows


In [21]:
cursor = conn.cursor()
cursor.execute("""
SELECT CustomerID, COUNT(*) FROM OrderDetail GROUP BY CustomerID LIMIT 5;
""")
print(cursor.fetchall())
cursor.close()


[(1, 7098), (2, 6220), (3, 7047), (4, 7422), (5, 7355)]
